# Movie Posters

## Dataset preparation
Since the text dataset is stupidly formatted, we need to run the following command:

```bash
for file in *; do iconv -f `file -I $file | sed -E 's/.+charset\=(.+)/\1/g'` -t utf-8 $file | sed -E 's/ObjectId\((".+")\)/\1/g' > ../fixed/$file; done
for file in *; do echo $file && cat $file | jq -s > ../$file.json; done
cat *.json | jq -s ".[][]" | jq -s  > all.json
```

In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import sys
sys.path.append('../')

from utils import repeat_n_times, get_latex_table
from IPython.display import display

import torch
import numpy as np
from glob import glob
import pandas as pd

In [3]:
# Hyperparameters
num_repetitions = 5
include_models = ["ae"]

# Creating embeddings

In [4]:
# image_paths = sorted(glob("../data/movie_genres/Movie_Poster_Dataset/*/*.jpg"))
# image_ids = [p.split("/")[-1].replace(".jpg", "") for p in image_paths]

# create_image_embeddings(image_paths, image_ids, "../data/movie_genres/image_features.pt")

In [5]:
# load image features and labels
image_features = torch.load("../data/movie_genres/image_features.pt")
df = pd.read_json('../data/movie_genres/all.json')

In [6]:
len(df)

8873

In [7]:
len(image_features)

8052

In [8]:
df.drop_duplicates(subset=["imdbID"], inplace=True)
df = df[df["imdbID"].isin(image_features.keys())]
df.sort_values(by="imdbID", inplace=True)

In [9]:
image_features = [image_features[i] for i in sorted(image_features.keys())]
image_features = torch.tensor(np.stack(image_features, axis=0))

In [10]:
image_features.shape

torch.Size([8052, 512])

In [11]:
len(df)

8052

# Experiments

## Movie Genre

In [12]:
labels = df["Genre"].tolist()
labels = [l.split(",")[0] for l in labels]

In [13]:
pd.Series(labels).value_counts()

Comedy         2409
Drama          1565
Action         1392
Documentary     729
Crime           444
Adventure       359
Biography       336
Animation       264
Horror          249
Fantasy          60
Thriller         46
Short            42
Mystery          33
Romance          31
Family           28
Sci-Fi           17
Musical          12
N/A              11
Music             8
Western           7
War               5
History           2
Sport             1
News              1
Reality-TV        1
dtype: int64

In [14]:
texts = [f"a poster of a {m} movie" for m in set(labels)]

means, stds = repeat_n_times(num_repetitions, labels, image_features, texts=texts, include_models=include_models)

Creating text features
Run 1
Getting Autoencoder performance


Loss (5729): 0.028293369337916374: : 5729it [01:44, 54.96it/s]


Finished optimization. Best loss (0.0197127778083086) achieved after 5629 iterations.
Run 2
Getting Autoencoder performance


Loss (759): 1.562257170677185: : 759it [00:14, 51.48it/s] 


Finished optimization. Best loss (1.5612962245941162) achieved after 659 iterations.
Run 3
Getting Autoencoder performance


...

Finished optimization. Best loss (0.017924601212143898) achieved after 2576 iterations.
Run 4
Getting Autoencoder performance


Loss (3975): 0.05168307200074196: : 3975it [01:13, 53.91it/s] 


Finished optimization. Best loss (0.04423704743385315) achieved after 3875 iterations.
Run 5
Getting Autoencoder performance


Loss (3947): 0.03203444927930832: : 3947it [01:24, 46.88it/s] 


Finished optimization. Best loss (0.027630770578980446) achieved after 3847 iterations.


In [15]:
display(means)
display(stds)

,AE
AMI,0.106980
NMI,0.117967
mean_average_precision,0.242061
mean_average_precision_at_r,0.098087
mean_reciprocal_rank,0.494694
precision_at_1,0.333259
r_precision,0.248285


,AE
AMI,0.043741
NMI,0.043212
mean_average_precision,0.027987
mean_average_precision_at_r,0.024394
mean_reciprocal_rank,0.026992
precision_at_1,0.030370
r_precision,0.030701


In [16]:
print(get_latex_table(means, stds, "Movie Posters, Genre", "fig:movie_posters_genre"))

\begin{table}
\centering
\caption{Movie Posters, Genre}
\label{fig:movie_posters_genre}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.107 $\pm$ 0.044 \\
\textbf{NMI   } &  0.118 $\pm$ 0.043 \\
\textbf{MAP   } &  0.242 $\pm$ 0.028 \\
\textbf{MAP@R } &  0.098 $\pm$ 0.024 \\
\textbf{MRR   } &  0.495 $\pm$ 0.027 \\
\textbf{Prec@1} &  0.333 $\pm$ 0.030 \\
\textbf{R-Prec} &  0.248 $\pm$ 0.031 \\
\bottomrule
\end{tabular}
\end{table}



/Users/konstantinkobs/code/2022-dml-clip/experiments/../utils.py:215: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return final_results.to_latex(bold_rows=True, escape=False, caption=caption, label=label, column_format=columns_format)


In [17]:
print("### Mean")
print(means.to_markdown())
print("### Std")
print(stds.to_markdown())

### Mean
|                             |       AE |
|:----------------------------|---------:|
| AMI                         | 0.10698  |
| NMI                         | 0.117967 |
| mean_average_precision      | 0.242061 |
| mean_average_precision_at_r | 0.098087 |
| mean_reciprocal_rank        | 0.494694 |
| precision_at_1              | 0.333259 |
| r_precision                 | 0.248285 |
### Std
|                             |        AE |
|:----------------------------|----------:|
| AMI                         | 0.0437407 |
| NMI                         | 0.0432118 |
| mean_average_precision      | 0.0279874 |
| mean_average_precision_at_r | 0.0243944 |
| mean_reciprocal_rank        | 0.0269919 |
| precision_at_1              | 0.0303698 |
| r_precision                 | 0.0307012 |


## Production Country

In [18]:
labels = df["Country"].tolist()
labels = [l.split(",")[0] for l in labels]

In [19]:
pd.Series(labels).value_counts()

USA                       6143
UK                         588
India                      319
France                     200
Canada                     183
                          ... 
Burkina Faso                 1
Dominican Republic           1
Bosnia and Herzegovina       1
Aruba                        1
Ukraine                      1
Length: 69, dtype: int64

In [20]:
texts = [f"a poster of a movie produced in {m}" for m in set(labels)]

means, stds = repeat_n_times(num_repetitions, labels, image_features, texts=texts, include_models=include_models)

Creating text features
Run 1
Getting Autoencoder performance


Loss (8092): 0.12250929325819016: : 8092it [03:28, 38.73it/s]


Finished optimization. Best loss (0.08888650685548782) achieved after 7992 iterations.
Run 2
Getting Autoencoder performance


Loss (4420): 0.2699509561061859: : 4420it [02:04, 35.63it/s] 


Finished optimization. Best loss (0.21542739868164062) achieved after 4320 iterations.
Run 3
Getting Autoencoder performance


...

Finished optimization. Best loss (0.20913584530353546) achieved after 4487 iterations.
Run 4
Getting Autoencoder performance


Loss (5609): 0.16797733306884766: : 5609it [02:38, 35.46it/s]


Finished optimization. Best loss (0.09635263681411743) achieved after 5509 iterations.
Run 5
Getting Autoencoder performance


Loss (6171): 0.21248885989189148: : 6171it [02:37, 39.17it/s]


Finished optimization. Best loss (0.20219622552394867) achieved after 6071 iterations.


In [21]:
display(means)
display(stds)

,AE
AMI,0.046727
NMI,0.090816
mean_average_precision,0.628757
mean_average_precision_at_r,0.493525
mean_reciprocal_rank,0.739011
precision_at_1,0.649240
r_precision,0.623929


,AE
AMI,0.007248
NMI,0.006848
mean_average_precision,0.005327
mean_average_precision_at_r,0.007289
mean_reciprocal_rank,0.004634
precision_at_1,0.006804
r_precision,0.005511


In [22]:
print(get_latex_table(means, stds, "Movie Posters, Production Country", "fig:movie_posters_production_country"))

\begin{table}
\centering
\caption{Movie Posters, Production Country}
\label{fig:movie_posters_production_country}
\begin{tabular}{@{}rc@{}}
\toprule
{} &                 AE \\
\midrule
\textbf{AMI   } &  0.047 $\pm$ 0.007 \\
\textbf{NMI   } &  0.091 $\pm$ 0.007 \\
\textbf{MAP   } &  0.629 $\pm$ 0.005 \\
\textbf{MAP@R } &  0.494 $\pm$ 0.007 \\
\textbf{MRR   } &  0.739 $\pm$ 0.005 \\
\textbf{Prec@1} &  0.649 $\pm$ 0.007 \\
\textbf{R-Prec} &  0.624 $\pm$ 0.006 \\
\bottomrule
\end{tabular}
\end{table}



/Users/konstantinkobs/code/2022-dml-clip/experiments/../utils.py:215: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return final_results.to_latex(bold_rows=True, escape=False, caption=caption, label=label, column_format=columns_format)


In [23]:
print("### Mean")
print(means.to_markdown())
print("### Std")
print(stds.to_markdown())

### Mean
|                             |        AE |
|:----------------------------|----------:|
| AMI                         | 0.0467268 |
| NMI                         | 0.0908159 |
| mean_average_precision      | 0.628757  |
| mean_average_precision_at_r | 0.493525  |
| mean_reciprocal_rank        | 0.739011  |
| precision_at_1              | 0.64924   |
| r_precision                 | 0.623929  |
### Std
|                             |         AE |
|:----------------------------|-----------:|
| AMI                         | 0.00724767 |
| NMI                         | 0.00684797 |
| mean_average_precision      | 0.00532681 |
| mean_average_precision_at_r | 0.00728949 |
| mean_reciprocal_rank        | 0.00463366 |
| precision_at_1              | 0.00680438 |
| r_precision                 | 0.0055108  |
